In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_24weeks_hf_male = sc.read_h5ad('../../exported_data/pvat_24weeks_hf_male.h5ad')
pvat_24weeks_hf_male

AnnData object with n_obs × n_vars = 13385 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered',

In [3]:
pvat_24weeks_hf_male.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Endothelial_Cells',
 'Immune_Cells',
 'Fibroblasts',
 'Mesothelial_Cells',
 'Pericytes',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_24weeks_hf_male.raw = pvat_24weeks_hf_male
pvat_24weeks_hf_male.raw.X

<13385x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 20296333 stored elements in Compressed Sparse Row format>

In [6]:
# Run cellchat analysis on PVAT data
cellchat(pvat_24weeks_hf_male, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellchat_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
966 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.18 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 13385 samples and 19777 features


100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:40<00:00,  4.54it/s]


In [7]:
ligand_receptor_pvat = pvat_24weeks_hf_male.uns['cellchat_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
10501,Sorbs1,Sorbs1,0.979381,0.028846,416.0,Itga1,Itga1,0.808511,0.006611,Adipocytes,SMCs,0.000381,0.0
89,Vegfa,Vegfa,0.943075,0.013221,416.0,Nrp1,Nrp1,0.806029,0.011418,Adipocytes,Adipocytes,0.000302,0.0
11726,Ncam1,Ncam1,0.875000,0.004507,416.0,Cacna1c,Cacna1c,0.957447,0.030649,Neuronal_Cells,SMCs,0.000276,0.0
10947,Lama2,Lama2,0.935484,0.018029,416.0,Itga1,Itga1,0.808511,0.006611,Fibroblasts,SMCs,0.000238,0.0
10483,Vegfa,Vegfa,0.943075,0.013221,416.0,Itgb1,Itgb1,0.744681,0.006310,Adipocytes,SMCs,0.000167,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellchat_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals] 

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
307,Col3a1,Col3a1,0.750538,0.006010,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,7.223043e-06,0.0
270,Col4a1,Col4a1,0.652688,0.005409,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,6.500744e-06,0.0
301,Col5a3,Col5a3,0.688172,0.004808,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,5.778443e-06,0.0
359,Col4a2,Col4a2,0.609677,0.003606,416.0,Itgb5,Itgb5,0.273196,0.000601,Fibroblasts,Adipocytes,4.333838e-06,0.0
276,Col4a2,Col4a2,0.609677,0.003606,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,4.333838e-06,0.0
280,Col6a2,Col6a2,0.656989,0.003005,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,3.611535e-06,0.0
264,Col6a6,Col6a6,0.526882,0.003005,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,3.611535e-06,0.0
303,Col5a2,Col5a2,0.669892,0.003005,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,3.611535e-06,0.0
278,Col6a1,Col6a1,0.539785,0.002404,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,2.889230e-06,0.0
282,Col6a3,Col6a3,0.559140,0.002404,416.0,Itgb1,Itga1_Itgb1,0.304684,0.000601,Fibroblasts,Adipocytes,2.889230e-06,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_24weeks_hf_male_cellchat.csv")
pvat_24weeks_hf_male.write("../../exported_data/pvat_24weeks_hf_male.h5ad")